In [1]:
#ssh sydney
#setPython3 

import numpy 
import numpy as np
import math
from numpy import loadtxt
import pandas as pd

from scipy.io import arff
data = arff.loadarff('jura.arff')
df = pd.DataFrame(data[0])

covariables = df.iloc[:,0:15].as_matrix()
response = df.iloc[:,15:18].as_matrix()
positions = np.arange(359)

from sklearn.model_selection import train_test_split


covariables_train, covariables_test, response_train, response_test,positions_train,positions_test = train_test_split(covariables, response,positions, test_size=0.20, random_state=42)


##APPRENTISSAGE

def shuffle_in_unison(a, b):
    assert len(a) == len(b)
    shuffled_a = numpy.empty(a.shape, dtype=a.dtype)
    shuffled_b = numpy.empty(b.shape, dtype=b.dtype)
    permutation = numpy.random.permutation(len(a))
    for old_index, new_index in enumerate(permutation):
        shuffled_a[new_index] = a[old_index]
        shuffled_b[new_index] = b[old_index]
    return shuffled_a, shuffled_b

import tensorflow as tf 




# Import data
# Tensorflow is finicky about shapes, so resize


X_datatrain, X_datatest, targets_train, targets_test = covariables_train, covariables_test, response_train, response_test

#les dimensions de X for training
NbrLignes_train = X_datatrain.shape[0]  ###
NbrColonnes_train= X_datatrain.shape[1] ###

#les dimensions de X for testing
NbrLignes_test = X_datatest.shape[0]  ###
NbrColonnes_test = X_datatest.shape[1] ###
#Y_data = targets 

Y_datatrain = tf.reshape(targets_train, [NbrLignes_train,targets_train.shape[1]])
Y_datatest = tf.reshape(targets_test, [NbrLignes_test,targets_train.shape[1]])

#build the model

# input X: batch_size x NbrColonnes, the first dimension (None) will index the data in the mini-batch
Xfill = tf.placeholder(tf.float32, shape= [None, NbrColonnes_train])
# correct answers will go here
Yfill = tf.placeholder(tf.float32, shape=[None, targets_train.shape[1]]) 
#Let's add some layers

# Probability of keeping a node during dropout = 1.0 at test time (no dropout) and 0.75 at training time
pkeep = tf.placeholder(tf.float32)

# five layers and their number of neurons (tha last layer has 10 softmax neurons)
L = 200
M = 160
N = 100
O = 30
# Weights initialised with small random values between -0.2 and +0.2
# When using RELUs, make sure biases are initialised with small *positive* values for example 0.1 = tf.ones([K])/10
W1 = tf.Variable(tf.truncated_normal([NbrColonnes_train, L], stddev=0.1))  # 
B1 = tf.Variable(tf.ones([L])/10)
W2 = tf.Variable(tf.truncated_normal([L, M], stddev=0.1))
B2 = tf.Variable(tf.ones([M])/10)
W3 = tf.Variable(tf.truncated_normal([M, N], stddev=0.1))
B3 = tf.Variable(tf.ones([N])/10)
W4 = tf.Variable(tf.truncated_normal([N, O], stddev=0.1))
B4 = tf.Variable(tf.ones([O])/10)
W5 = tf.Variable(tf.truncated_normal([O,targets_train.shape[1]], stddev=0.1))
B5 = tf.Variable(tf.zeros([targets_train.shape[1]]))

# The model, with dropout at each layer


Y1 = tf.nn.relu(tf.matmul(Xfill, W1) + B1)
Y1d = tf.nn.dropout(Y1, pkeep)

Y2 = tf.nn.relu(tf.matmul(Y1d, W2) + B2)
Y2d = tf.nn.dropout(Y2, pkeep)

Y3 = tf.nn.relu(tf.matmul(Y2d, W3) + B3)
Y3d = tf.nn.dropout(Y3, pkeep)

Y4 = tf.nn.relu(tf.matmul(Y3d, W4) + B4)
Y4d = tf.nn.dropout(Y4, pkeep)

Y_ = tf.matmul(Y4d, W5) + B5

cross_entropy = tf.losses.mean_squared_error(Yfill, Y_)

# accuracy of the trained model, between 0 (worst) and 1 (best)
#Modification de l'accuracy 

#calcul de l'accuracy, une nouvelle manière: :  

# variable learning rate
lr = tf.placeholder(tf.float32)
train_step = tf.train.AdamOptimizer(lr).minimize(cross_entropy) 

# init
init = tf.global_variables_initializer()
sess = tf.InteractiveSession()
sess.run(init)


max_learning_rate = 0.003
min_learning_rate = 0.0001
decay_speed = 100.0 # 0.003-0.0001-2000=>0.9826 done in 5000 iterations

train_loss = []
test_loss = []

# Gradient descent loop for 500 steps
for i in range(3500):
 # Select random minibatch
    j = 1
    ##### shuffling our data::: really important
    X_datasf, Y_datasf = shuffle_in_unison(X_datatrain, Y_datatrain.eval())
    
    ##### defining a decreasing learning rate: 
    # learning rate decay
    learning_rate = 0.0001001329829992624
    #learning_rate = min_learning_rate + (max_learning_rate - min_learning_rate) * math.exp(-i/decay_speed)
    
    ##### 
    for start, end in zip(range(0, NbrLignes_train, 52), range(20, NbrLignes_train, 52)):

             X_batch,Y_batch = X_datasf[start:end,],Y_datasf[start:end,]
             j +=1
             # Do gradient descent step
             _, loss_val_train = sess.run([train_step, cross_entropy], feed_dict={Xfill: X_batch, Yfill: Y_batch, pkeep: 0.75, lr: learning_rate})
             if i %20==0:    
                 
             
                #if j%20==0:
                 loss_val_test = sess.run(cross_entropy, feed_dict={Xfill: X_datatest, Yfill: Y_datatest.eval(), pkeep: 1.0, lr:  learning_rate})
                 train_loss.append(loss_val_train)
                 
                 test_loss.append(loss_val_test)
                 print('LOSS at step %s: - Train loss value : %f -- Test loss value : %f' % (i, loss_val_train,loss_val_test))
#print(test_loss)
#print(test_loss)
#LOSS at step 420: - Train loss value : 22.385496 -- Test loss value : 9.721767

C:\Users\Trader\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
C:\Users\Trader\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  from ipykernel import kernelapp as app


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
LOSS at step 0: - Train loss value : 317.996002 -- Test loss value : 266.846649
LOSS at step 0: - Train loss value : 196.896973 -- Test loss value : 263.868744
LOSS at step 0: - Train loss value : 229.646286 -- Test loss value : 260.912354
LOSS at step 0: - Train loss value : 379.630280 -- Test loss value : 258.245605
LOSS at step 0: - Train loss value : 914.169800 -- Test loss value : 256.049683
LOSS at step 0: - Train loss value : 394.048492 -- Test loss value : 253.943634
LOSS at step 20: - Train loss value : 130.232376 -- Test loss value : 55.534424
LOSS at step 20: - Train loss value : 99.174850 -- Test loss value : 54.728252
LOSS at step 20: - Train loss value : 197.552231 -- Test loss value : 53.914188
LOSS at step 20: - Train loss value : 126.489304 -- Test loss value : 53

LOSS at step 320: - Train loss value : 26.658810 -- Test loss value : 22.817492
LOSS at step 320: - Train loss value : 48.445450 -- Test loss value : 22.845278
LOSS at step 320: - Train loss value : 147.231033 -- Test loss value : 22.873083
LOSS at step 320: - Train loss value : 191.355743 -- Test loss value : 22.908005
LOSS at step 320: - Train loss value : 53.368244 -- Test loss value : 22.954599
LOSS at step 320: - Train loss value : 26.036222 -- Test loss value : 23.009434
LOSS at step 340: - Train loss value : 185.785675 -- Test loss value : 23.439512
LOSS at step 340: - Train loss value : 76.218979 -- Test loss value : 23.355984
LOSS at step 340: - Train loss value : 29.174858 -- Test loss value : 23.306482
LOSS at step 340: - Train loss value : 31.818443 -- Test loss value : 23.276573
LOSS at step 340: - Train loss value : 34.198284 -- Test loss value : 23.254923
LOSS at step 340: - Train loss value : 54.106976 -- Test loss value : 23.227978
LOSS at step 360: - Train loss value 

LOSS at step 680: - Train loss value : 46.300182 -- Test loss value : 20.584915
LOSS at step 680: - Train loss value : 62.776001 -- Test loss value : 20.634983
LOSS at step 680: - Train loss value : 69.310661 -- Test loss value : 20.679399
LOSS at step 680: - Train loss value : 198.782288 -- Test loss value : 20.664181
LOSS at step 680: - Train loss value : 46.231152 -- Test loss value : 20.668833
LOSS at step 680: - Train loss value : 28.327759 -- Test loss value : 20.673698
LOSS at step 700: - Train loss value : 43.945606 -- Test loss value : 21.037479
LOSS at step 700: - Train loss value : 17.639282 -- Test loss value : 21.045645
LOSS at step 700: - Train loss value : 92.076447 -- Test loss value : 21.031591
LOSS at step 700: - Train loss value : 47.323502 -- Test loss value : 21.028852
LOSS at step 700: - Train loss value : 153.659561 -- Test loss value : 21.058584
LOSS at step 700: - Train loss value : 62.836727 -- Test loss value : 21.086508
LOSS at step 720: - Train loss value :

LOSS at step 1040: - Train loss value : 94.322197 -- Test loss value : 22.863802
LOSS at step 1040: - Train loss value : 103.033844 -- Test loss value : 22.938599
LOSS at step 1040: - Train loss value : 37.309658 -- Test loss value : 23.006212
LOSS at step 1040: - Train loss value : 22.702009 -- Test loss value : 23.079403
LOSS at step 1040: - Train loss value : 21.722746 -- Test loss value : 23.167870
LOSS at step 1040: - Train loss value : 12.935975 -- Test loss value : 23.272781
LOSS at step 1060: - Train loss value : 38.359425 -- Test loss value : 24.804186
LOSS at step 1060: - Train loss value : 55.597034 -- Test loss value : 24.676418
LOSS at step 1060: - Train loss value : 9.460901 -- Test loss value : 24.568010
LOSS at step 1060: - Train loss value : 11.571671 -- Test loss value : 24.479664
LOSS at step 1060: - Train loss value : 79.050423 -- Test loss value : 24.398226
LOSS at step 1060: - Train loss value : 115.026299 -- Test loss value : 24.369873
LOSS at step 1080: - Train 

...

LOSS at step 2060: - Train loss value : 116.638741 -- Test loss value : 23.532337
LOSS at step 2060: - Train loss value : 79.217964 -- Test loss value : 23.670521
LOSS at step 2060: - Train loss value : 33.488827 -- Test loss value : 23.803385
LOSS at step 2060: - Train loss value : 17.992388 -- Test loss value : 23.920143
LOSS at step 2060: - Train loss value : 68.732407 -- Test loss value : 23.960354
LOSS at step 2060: - Train loss value : 33.877697 -- Test loss value : 23.967154
LOSS at step 2080: - Train loss value : 26.015863 -- Test loss value : 23.204859
LOSS at step 2080: - Train loss value : 75.127304 -- Test loss value : 23.117735
LOSS at step 2080: - Train loss value : 20.893679 -- Test loss value : 23.111948
LOSS at step 2080: - Train loss value : 29.441031 -- Test loss value : 23.081478
LOSS at step 2080: - Train loss value : 27.828129 -- Test loss value : 23.049484
LOSS at step 2080: - Train loss value : 92.152481 -- Test loss value : 23.046091
LOSS at step 2100: - Train 

LOSS at step 2400: - Train loss value : 8.470183 -- Test loss value : 26.142397
LOSS at step 2400: - Train loss value : 17.858381 -- Test loss value : 26.130983
LOSS at step 2400: - Train loss value : 95.433197 -- Test loss value : 26.083902
LOSS at step 2400: - Train loss value : 18.142145 -- Test loss value : 26.009569
LOSS at step 2400: - Train loss value : 180.585693 -- Test loss value : 25.892393
LOSS at step 2400: - Train loss value : 20.635914 -- Test loss value : 25.773752
LOSS at step 2420: - Train loss value : 43.579964 -- Test loss value : 22.578762
LOSS at step 2420: - Train loss value : 14.299062 -- Test loss value : 22.492458
LOSS at step 2420: - Train loss value : 28.899494 -- Test loss value : 22.428793
LOSS at step 2420: - Train loss value : 58.742783 -- Test loss value : 22.327686
LOSS at step 2420: - Train loss value : 25.492537 -- Test loss value : 22.160269
LOSS at step 2420: - Train loss value : 12.787395 -- Test loss value : 21.980154
LOSS at step 2440: - Train l

LOSS at step 2740: - Train loss value : 40.583961 -- Test loss value : 28.402029
LOSS at step 2740: - Train loss value : 51.700954 -- Test loss value : 28.199602
LOSS at step 2740: - Train loss value : 18.349422 -- Test loss value : 27.982504
LOSS at step 2740: - Train loss value : 19.727451 -- Test loss value : 27.807055
LOSS at step 2740: - Train loss value : 66.403244 -- Test loss value : 27.699389
LOSS at step 2740: - Train loss value : 100.314568 -- Test loss value : 27.696012
LOSS at step 2760: - Train loss value : 40.378643 -- Test loss value : 24.188969
LOSS at step 2760: - Train loss value : 207.654495 -- Test loss value : 24.283756
LOSS at step 2760: - Train loss value : 96.059898 -- Test loss value : 24.362207
LOSS at step 2760: - Train loss value : 37.857502 -- Test loss value : 24.400698
LOSS at step 2760: - Train loss value : 31.625040 -- Test loss value : 24.419155
LOSS at step 2760: - Train loss value : 23.796930 -- Test loss value : 24.434654
LOSS at step 2780: - Train

LOSS at step 3080: - Train loss value : 65.811409 -- Test loss value : 28.180002
LOSS at step 3080: - Train loss value : 38.450584 -- Test loss value : 28.087389
LOSS at step 3080: - Train loss value : 21.839237 -- Test loss value : 28.030176
LOSS at step 3080: - Train loss value : 77.932175 -- Test loss value : 28.102345
LOSS at step 3080: - Train loss value : 22.505136 -- Test loss value : 28.113472
LOSS at step 3080: - Train loss value : 52.122265 -- Test loss value : 28.198305
LOSS at step 3100: - Train loss value : 34.403938 -- Test loss value : 25.808712
LOSS at step 3100: - Train loss value : 15.484036 -- Test loss value : 25.766035
LOSS at step 3100: - Train loss value : 23.764587 -- Test loss value : 25.732859
LOSS at step 3100: - Train loss value : 17.934650 -- Test loss value : 25.715702
LOSS at step 3100: - Train loss value : 35.196289 -- Test loss value : 25.740452
LOSS at step 3100: - Train loss value : 76.972237 -- Test loss value : 25.817112
LOSS at step 3120: - Train l

LOSS at step 3420: - Train loss value : 22.899126 -- Test loss value : 25.550795
LOSS at step 3420: - Train loss value : 19.739269 -- Test loss value : 25.507545
LOSS at step 3420: - Train loss value : 181.960037 -- Test loss value : 25.607126
LOSS at step 3420: - Train loss value : 143.995911 -- Test loss value : 25.739779
LOSS at step 3420: - Train loss value : 12.425731 -- Test loss value : 25.849281
LOSS at step 3420: - Train loss value : 38.701801 -- Test loss value : 25.945456
LOSS at step 3440: - Train loss value : 194.998489 -- Test loss value : 25.191374
LOSS at step 3440: - Train loss value : 27.864286 -- Test loss value : 25.284191
LOSS at step 3440: - Train loss value : 31.236866 -- Test loss value : 25.367468
LOSS at step 3440: - Train loss value : 29.034893 -- Test loss value : 25.425676
LOSS at step 3440: - Train loss value : 28.467085 -- Test loss value : 25.473095
LOSS at step 3440: - Train loss value : 35.798740 -- Test loss value : 25.523790
LOSS at step 3460: - Trai

# Let us compute the aRRMSE

In [2]:
learning_rate = 0.0001001329829992624

Y_testpred = Y_.eval(feed_dict={Xfill: tf.cast(X_datatest, tf.float32).eval(), Yfill: tf.cast(Y_datatest, tf.float32).eval(), pkeep: 1.0, lr: learning_rate})
#numpy.savetxt('Y_testpred_with_stock_%s_and_ai_%s_modelcut_%s_vartimeframe_%s.txt'%(stock, ai, cut, vartimeframe), Y_testpred, fmt='%f')
Y_trainpred = Y_.eval(feed_dict={Xfill: tf.cast(X_datatrain, tf.float32).eval(), Yfill: tf.cast(Y_datatrain, tf.float32).eval(), pkeep: 1.0, lr: learning_rate})


In [3]:
Y_testpred

array([[  1.457536  ,  10.6360655 ,  17.0774    ],
       [  1.3593664 ,   9.407509  ,  12.937172  ],
       [  1.2807195 ,   9.2058    ,  16.304344  ],
       [  0.6323885 ,   3.4691236 ,  20.188906  ],
       [  1.4730468 ,  10.686439  ,  14.641312  ],
       [  1.1512225 ,   7.3581905 ,   6.230218  ],
       [  0.8084021 ,   4.9692626 ,  12.666358  ],
       [  1.5001407 ,  11.102488  ,  19.501356  ],
       [  1.4121099 ,   9.66781   ,  10.406562  ],
       [  1.2851796 ,  10.27781   ,  57.51046   ],
       [  0.8989567 ,   5.566701  ,   7.5953526 ],
       [  1.3590796 ,   9.591845  ,  12.812217  ],
       [  1.252224  ,   8.539697  ,  10.396381  ],
       [  1.3462628 ,   9.39685   ,  12.046166  ],
       [  1.1855993 ,   8.061473  ,  12.104553  ],
       [  1.6813955 ,  12.77926   ,  21.662706  ],
       [  1.6216304 ,  11.662506  ,  11.981698  ],
       [  0.67995954,   3.8517027 ,  22.509413  ],
       [  1.0148169 ,   7.4513793 ,  27.487257  ],
       [  1.0569217 ,   6.62480

In [4]:
from numpy import linalg as LA

In [5]:

aRRMSE_list = []

for i in range(Y_testpred.shape[1]) :
    
    Errors_test =  Y_testpred[:,i] - response_test[:,i]
    print('<*********************************Let us check the aRRMSE*********************************>')

    ytrain_mean_i = np.tile(Y_trainpred[:,i].mean(axis = 0), (covariables_test.shape[0], 1))

    Errors_relative = ytrain_mean_i - response_test[:,i]

    Final_SPNNR_aRRMSE = 100*np.sqrt((LA.norm(Errors_test)**2)/(LA.norm(Errors_relative)**2))  

    print("Final i-th aRRMSE is : %f"%Final_SPNNR_aRRMSE)

    aRRMSE_list.append(Final_SPNNR_aRRMSE)

<*********************************Let us check the aRRMSE*********************************>
Final i-th aRRMSE is : 10.299022
<*********************************Let us check the aRRMSE*********************************>
Final i-th aRRMSE is : 8.515590
<*********************************Let us check the aRRMSE*********************************>
Final i-th aRRMSE is : 5.648499


In [6]:
np.mean(aRRMSE_list)

8.154370066890465

In [9]:
#Taking aRRMSE on PCA or inverted PCA. ?

# Check if old way calculation is wrong

In [10]:
Errors_test =  Y_testpred - response_test


print('<*********************************Let us check the aRRMSE*********************************>')

ytrain_mean = np.tile(Y_trainpred.mean(axis = 0), (response_test.shape[0], 1))

Errors_relative = ytrain_mean - response_test

Final_SPNNR_aRRMSE = 100*np.sqrt((LA.norm(Errors_test)**2)/(LA.norm(Errors_relative)**2))  

print("Final_Partition aRRMSE is : %f"%Final_SPNNR_aRRMSE)

<*********************************Let us check the aRRMSE*********************************>
Final_Partition aRRMSE is : 82.647642


*** The answer is yes ***

# Checking True aRRMSE of S-SPNNR on Slump

In [11]:
spnnr_covariables_train, spnnr_covariables_test, spnnr_response_train, spnnr_response_test,spnnr_positions_train,spnnr_positions_test = train_test_split(covariables, response,positions, test_size=0.33, random_state=42)

In [12]:
spnnr_Y_testpred =np.loadtxt('All_Y_test_predictions.out', delimiter=',' )
spnnr_Y_trainpred =np.loadtxt('All_Y_train_predictions.out', delimiter=',' )

In [10]:
spnnr_aRRMSE_list = []

for i in range(spnnr_Y_testpred.shape[1]) :
    
    spnnr_Errors_test =  spnnr_Y_testpred[:,i] - spnnr_response_test[:,i]
    print('<*********************************Let us check the aRRMSE*********************************>')

    spnnr_ytrain_mean_i = np.tile(spnnr_Y_trainpred[:,i].mean(axis = 0), (spnnr_covariables_test.shape[0], 1))

    spnnr_Errors_relative = spnnr_ytrain_mean_i - spnnr_response_test[:,i]

    spnnr_Final_SPNNR_aRRMSE = 100*np.sqrt((LA.norm(spnnr_Errors_test)**2)/(LA.norm(spnnr_Errors_relative)**2))  

    print("Final i-th aRRMSE is : %f"%spnnr_Final_SPNNR_aRRMSE)

    spnnr_aRRMSE_list.append(spnnr_Final_SPNNR_aRRMSE)

<*********************************Let us check the aRRMSE*********************************>
Final i-th aRRMSE is : 9.212108
<*********************************Let us check the aRRMSE*********************************>
Final i-th aRRMSE is : 17.784301
<*********************************Let us check the aRRMSE*********************************>
Final i-th aRRMSE is : 18.862991


In [11]:
np.mean(spnnr_aRRMSE_list)

15.286466834678842